In [12]:
import pandas as pd
from tqdm import tqdm

In [13]:
#!python -m spacy download es_core_news_lg

In [92]:
df = pd.read_csv('./final_bd_cel_ner_all.csv')
df.head()

,Unnamed: 0,Encrypted_Seller,Encrypted_Id,Title,marca,start,end
0,0,7a30f2b511ff323b31dc01de07736128b49d5ec4c50b01...,054dd4f1afba59722078d8a4af2f1b3742d767e836ace2...,Teléfono Nokia 1600,Nokia,9,14
1,1,f6fed6d5cf9a02286d81f9c842a3874511ff1d3d411a94...,6c6a089073a7ba0ba6897eb51b0a0b5dc50d8071edafef...,Celular Motorola G6,Motorola,8,16
2,2,eef17e4b9fa3ccc0232a39061e31e721f5ca6a7a44b786...,78364fc1d8574957fe3508ed687d862a0bbcf5e5136558...,iPhone 6 Plus 16gb,iPhone,0,6
3,3,2ab9d5c1ec1dc9cf166fd20e51a3e734ab5076a4e5c73d...,896a81cc2da0ecb382e3b519783a8ef2d9ee0ce3872ea3...,iPhone 8 64 Gb Space Gray,iPhone,1,7
4,4,0f9b41b18f3c3776dc9515e1eb2a8686d4b271341444bb...,e6733e9d8b3bd38642d841c4509b0a5d6b5545d9a75fab...,Huawei Mate 10 Lite,Huawei,0,6


In [77]:
marcas = pd.read_csv('./marcas_meli.csv')
marcas.head()

,marca
0,Acer
1,Alcatel
2,Apple
3,iPhone
4,BlackBerry


In [26]:
import spacy
#from spacy import displacy

nlp = spacy.load('es_core_news_lg', disable = ['tagger','ner'])

In [44]:
#Vectores Marcas
list_vec = marcas['marca'].to_list()
marcas_vec = [nlp(x.lower()) for x in list_vec]

In [45]:
# Vectores NER
list_vec = df['marca'].to_list()
ner_vec = [nlp(x.lower()) for x in list_vec]

In [46]:
clean_ents = []

for ner in ner_vec:
    paso = 0
    for marca in marcas_vec:
        #print("Lista ruler: ", vec)
        similarity = marca.similarity(ner)
        #print(similarity)
        #print("Lista ner: ", text1)
        #print("........")                                        
        if similarity > 0.90:
            #print(marca.text)
            clean_ents.append(marca.text)
            paso = 1
            break
    if paso == 0:
        clean_ents.append(ner.text)    


/tmp/ipykernel_7011/1658257045.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = marca.similarity(ner)


In [47]:
pd.Series(clean_ents).value_counts()

iphone       29206
samsung      20223
##rev         7619
motorola      7192
xiaomi        4605
             ...  
s6edge           1
samsungj6        1
morotola         1
carretel         1
samguns          1
Length: 313, dtype: int64

In [48]:
pd.Series(list_vec).value_counts()

iPhone        29206
Samsung       20219
##REV          7619
Motorola       7187
Xiaomi         4604
              ...  
Venus             1
Soni              1
Samsungj6         1
Retro             1
Samsunga51        1
Length: 322, dtype: int64

In [50]:
text = "iPhone"
text1 = "i phone"
text_nlp = nlp(text)
text1_nlp = nlp(text1)
text_nlp.similarity(text1_nlp)


0.21120255842624355

# Fastext + Spacy

In [75]:
!python -m spacy init-model es '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/cel/models/cel_model_vec' --vectors-loc '/home/fede/fastText-0.9.2/result/nubi_cel_vectors_300.vec'

✔ Successfully created model
3067it [00:00, 32252.47it/s]/fede/fastText-0.9.2/result/nubi_cel_vectors_300.vec
✔ Loaded vectors from
/home/fede/fastText-0.9.2/result/nubi_cel_vectors_300.vec
✔ Sucessfully compiled vocab
3293 entries, 3067 vectors


In [98]:
nlp_cel = spacy.load("./models/cel_model_vec")
doc1 = nlp_cel("iPhone")
doc2 = nlp_cel("!phone")
doc1.similarity(doc2)

0.6308446180436149

In [92]:
tokens = nlp_cel("Zte")
for token in tokens:
    print(token.text, token.has_vector, token.vector_norm, token.is_oov)

Zte True 4.3635316 False


In [80]:
#Vectores Marcas
list_vec = marcas['marca'].to_list()
marcas_vec = [nlp_cel(x) for x in list_vec]

In [81]:
# Vectores NER
list_vec = df['marca'].to_list()
ner_vec = [nlp_cel(x) for x in list_vec]

In [82]:
clean_ents = []

for ner in ner_vec:
    paso = 0
    for marca in marcas_vec:
        #print("Lista ruler: ", vec)
        similarity = marca.similarity(ner)
        #print(similarity)
        #print("Lista ner: ", text1)
        #print("........")                                        
        if similarity > 0.90:
            #print(marca.text)
            clean_ents.append(marca.text)
            paso = 1
            break
    if paso == 0:
        clean_ents.append(ner.text)

/tmp/ipykernel_7011/3257275729.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = marca.similarity(ner)


In [83]:
pd.Series(clean_ents).value_counts()

iPhone         29206
Samsung        20577
Cubot           7657
Motorola        7207
Xiaomi          4698
               ...  
Smarthphone        1
Asegura            1
Poco               1
Venus              1
Samsunga51         1
Length: 286, dtype: int64

In [84]:
pd.Series(list_vec).value_counts()

iPhone        29206
Samsung       20219
##REV          7619
Motorola       7187
Xiaomi         4604
              ...  
Venus             1
Soni              1
Samsungj6         1
Retro             1
Samsunga51        1
Length: 322, dtype: int64

In [85]:
for token in marcas_vec:
    print(token.text, token.has_vector, token.vector_norm)

Acer True 2.216228506210208
Alcatel True 3.550143019269944
Apple True 3.567832890457819
BlackBerry True 0
Blackview True 4.050532550900071
BLU True 0
CAT True 0
Cubot True 1.361171144053608
Doogee True 2.5885041390662114
Google True 4.270662552062786
Huawei True 3.986888884382795
Hyundai True 3.1045789197594047
iPro True 0
Kodak True 4.57556179218432
Lenovo True 1.5968048777551884
LG True 3.999491753310649
Microsoft True 3.1962553611987463
Motorola True 3.674751220021149
Nokia True 3.295463415843772
OnePlus True 0
Quantum True 4.309247831419258
Realme True 3.010249306661398
Samsung True 3.8096327229586633
Sansei True 2.9731038029406127
Sky Devices True 2.6658457573403482
Sony True 3.9778093341609235
Sony Ericsson True 3.357867190182035
TCL True 0
Ulefone True 3.3164843880918116
Umidigi True 2.0079448101882034
Xiaomi True 3.6325875481544685
ZTE True 0


# Fastext

* %cd '/home/fede/fastText-0.9.2'
* !pip install fastText
* !./fasttext
* !./fasttext skipgram -input ./training_data/nubi_cel.jsonl -output models/fasttext/nubi_cel_vectors_300 -dim 300

In [1]:
import fasttext

In [98]:
model = fasttext.load_model('./models/fasttext/nubi_cel_vectors_300.bin')

In [99]:
fast_m = model.get_nearest_neighbors('Samsum', k=10)
fast_m

[(0.9964942932128906, 'Samsug'),
 (0.991128146648407, 'Samsun'),
 (0.9885378479957581, 'Samsumg'),
 (0.9852354526519775, 'Samsung,'),
 (0.9841498136520386, 'Samsung.'),
 (0.9749056100845337, 'Samsung'),
 (0.9736911058425903, 'Samaung'),
 (0.9712196588516235, 'Samgung'),
 (0.965300977230072, 'Sansung'),
 (0.9641319513320923, 'Samnsung')]

In [79]:
#Vectores Marcas
marcas = marcas['marca'].to_list()
ents = df['marca'].to_list()

In [80]:
marc_ents = []
count = 0
for ent in ents:
    paso = 0   
    
    marc = [x for x in marcas if ent.lower() in x.lower()]
    
    if marc:
        marc_ents.append(marc[0])
        paso = 1
    else:
        ent_neigs = model.get_nearest_neighbors(ent, k=10) 
        for ent_neig in ent_neigs:
            if ent_neig[0] > 0.95:
                marc_ = [x for x in marcas if ent_neig[1].lower() in x.lower()]            
                if marc_:
                    marc_ents.append(marc_[0])
                    #print(ent_neig[1])
                    paso = 1
                    break            
    if paso == 0:
        marc_ents.append(ent)
        count+=1
print(count)

9210


In [81]:
# new_list = [x for x in marcas if "Zte".lower() in x.lower()]
# new_list

In [87]:
pd.Series(marc_ents).value_counts()

iPhone      29517
Samsung     20617
##REV        7619
Motorola     7212
Xiaomi       4621
            ...  
Motoc           1
Mod             1
I'phone         1
Sonim           1
Selfie          1
Length: 234, dtype: int64

In [83]:
pd.Series(ents).value_counts()

iPhone             29206
Samsung            20219
##REV               7619
Motorola            7187
Xiaomi              4604
                   ...  
Woo                    1
Sm-a022mzklaro         1
Celular.samsung        1
Kyosera                1
Selfie                 1
Length: 322, dtype: int64

In [95]:
df['marcas_corregidas'] = marc_ents
df.loc[df['marcas_corregidas']=='iPhone', 'marcas_corregidas'] = 'Apple'
df.head()

,Unnamed: 0,Encrypted_Seller,Encrypted_Id,Title,marca,start,end,marcas_corregidas
0,0,7a30f2b511ff323b31dc01de07736128b49d5ec4c50b01...,054dd4f1afba59722078d8a4af2f1b3742d767e836ace2...,Teléfono Nokia 1600,Nokia,9,14,Nokia
1,1,f6fed6d5cf9a02286d81f9c842a3874511ff1d3d411a94...,6c6a089073a7ba0ba6897eb51b0a0b5dc50d8071edafef...,Celular Motorola G6,Motorola,8,16,Motorola
2,2,eef17e4b9fa3ccc0232a39061e31e721f5ca6a7a44b786...,78364fc1d8574957fe3508ed687d862a0bbcf5e5136558...,iPhone 6 Plus 16gb,iPhone,0,6,Apple
3,3,2ab9d5c1ec1dc9cf166fd20e51a3e734ab5076a4e5c73d...,896a81cc2da0ecb382e3b519783a8ef2d9ee0ce3872ea3...,iPhone 8 64 Gb Space Gray,iPhone,1,7,Apple
4,4,0f9b41b18f3c3776dc9515e1eb2a8686d4b271341444bb...,e6733e9d8b3bd38642d841c4509b0a5d6b5545d9a75fab...,Huawei Mate 10 Lite,Huawei,0,6,Huawei


In [96]:
df.to_csv("./cel_bd.csv", index=False)